In [1]:
# Загрузка библиотек
import numpy as np
import pandas as pd

In [2]:
# Загрузка двух баз данных
db1 = pd.read_excel('proc_db1.xlsx')
db2 = pd.read_excel('proc_db2.xlsx')

In [3]:
db1.head()

,Nanoparticle,coat,Diameter (nm),Concentration μM,Zeta potential (mV),Cells,Cell line (L)/primary cells (P),Human(H)/Animal(A) cells,Cell morphology,Cell-organ/tissue source,...,Test,Test indicator,Biochemical metric,% Cell viability,Interference checked (Y/N),Colloidal stability checked (Y/N),Positive control (Y/N),Animal,Type,Cell age
0,CeO2,nothing,20.3,0.002478,-8.0,L929,L,A,Fibroblast,Areolar tissue,...,MTT,tetrazolium salt,cell metabolic activity,100.00000,N,N,N,rat,1,A
1,CeO2,nothing,20.3,0.004956,-8.0,L929,L,A,Fibroblast,Areolar tissue,...,MTT,tetrazolium salt,cell metabolic activity,100.00000,N,N,N,rat,1,A
2,CeO2,nothing,20.3,0.009911,-8.0,L929,L,A,Fibroblast,Areolar tissue,...,MTT,tetrazolium salt,cell metabolic activity,100.00000,N,N,N,rat,1,A
3,CeO2,nothing,20.3,0.019823,-8.0,L929,L,A,Fibroblast,Areolar tissue,...,MTT,tetrazolium salt,cell metabolic activity,100.00000,N,N,N,rat,1,A
4,CeO2,nothing,20.3,0.039645,-8.0,L929,L,A,Fibroblast,Areolar tissue,...,MTT,tetrazolium salt,cell metabolic activity,97.53237,N,N,N,rat,1,A


In [4]:
db2.head()

,Material type,Elements,Electronegativity,Ionic radius,Core size (nm),Hydro size (nm),Surface charge (mV),Surface area (m2/g),Cell type,Exposure dose (ug/mL),Number of atoms,Molecular weight (g/mol),Topological polar surface area (Å²),Density (g/cm3),Viability (%)
0,CuO,Cu,1.9,73.0,12.8,313.8,7.6,74.2,1,200.0,2,79.545,17.1,5.94,9.1
1,CuO,Cu,1.9,73.0,12.8,313.8,7.6,74.2,1,100.0,2,79.545,17.1,5.94,14.3
2,CuO,Cu,1.9,73.0,12.8,305.3,7.6,74.2,0,200.0,2,79.545,17.1,5.94,14.8
3,CuO,Cu,1.9,73.0,12.8,313.8,7.6,74.2,1,50.0,2,79.545,17.1,5.94,16.7
4,CuO,Cu,1.9,73.0,12.8,305.3,7.6,74.2,0,100.0,2,79.545,17.1,5.94,23.9


In [5]:
# Переименование
db1.rename({
    'Type': 'Is inorganic'
}, inplace=True, axis=1)

In [6]:
db2['Material type'].unique()

array(['CuO', 'ZnO', 'Mn2O3', 'CoO', 'CeO2', 'Fe2O3', 'Gd2O3', 'HfO2',
       'In2O3', 'La2O3', 'NiO', 'Sb2O3', 'SiO2', 'Al2O3', 'Fe3O4', 'TiO2'],
      dtype=object)

In [7]:
# Добавление признака
db2['Is inorganic'] = np.ones(len(db2))

In [8]:
# Ещё признак
db2['Animal'] = ['unknown'] * len(db2)

In [9]:
# Вычисление концентрации
db2['Concentration μM'] = db2['Exposure dose (ug/mL)'] / db2['Molecular weight (g/mol)'] / 1000

In [10]:
# Переименование
db2.rename({
    'Surface area (m2/g)': 'Zeta potential (mV)',
    'Material type': 'Nanoparticle'
}, inplace=True, axis=1)

In [11]:
# Сопоставление
db1['Cell line (L)/primary cells (P)'].value_counts(normalize=True)

L    0.813536
P    0.186464
Name: Cell line (L)/primary cells (P), dtype: float64

In [110]:
db2['Cell type'].value_counts(normalize=True)

1    0.599061
0    0.400939
Name: Cell type, dtype: float64

In [13]:
# Приведение к одному формату
db2['Cell type'].replace({
    1: 'L',
    0: 'P'
}, inplace=True)

In [14]:
# Переименование...
db2.rename({
    'Cell type': 'Cell line (L)/primary cells (P)',
    'Hydro size (nm)': 'Diameter (nm)'
}, inplace=True, axis=1)

In [15]:
db1.rename({
    '% Cell viability': 'Viability (%)'
}, inplace=True, axis=1)

In [16]:
# Выявление общих столбцов 
cols = list(set(db1.columns) & set(db2.columns))
print(*cols, sep='\n')

Nanoparticle
Cell line (L)/primary cells (P)
Viability (%)
Is inorganic
Zeta potential (mV)
Concentration μM
Animal
Diameter (nm)


In [17]:
# Соединение...
df1 = db1.filter(cols)
df2 = db2.filter(cols)

df3 = pd.concat([df1, df2], axis=0)

In [18]:
df3.reset_index(inplace=True, drop=True)

In [19]:
df3.head()

,Nanoparticle,Cell line (L)/primary cells (P),Viability (%),Is inorganic,Zeta potential (mV),Concentration μM,Animal,Diameter (nm)
0,CeO2,L,100.00000,1.0,-8.0,0.002478,rat,20.3
1,CeO2,L,100.00000,1.0,-8.0,0.004956,rat,20.3
2,CeO2,L,100.00000,1.0,-8.0,0.009911,rat,20.3
3,CeO2,L,100.00000,1.0,-8.0,0.019823,rat,20.3
4,CeO2,L,97.53237,1.0,-8.0,0.039645,rat,20.3


In [20]:
# Сохранение БД
df3.to_excel('connected.xlsx', index=False)